In [ ]:
# Import python packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import shap

# ML Libraries
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from snowflake.ml.experiment import ExperimentTracking

# Feature store
from snowflake.ml.feature_store import FeatureStore
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Model registry
from snowflake.ml.registry import Registry
registry = Registry(session=session, database_name="dev", schema_name="data_science")

# Initialize Experiment Tracking
exp = ExperimentTracking(session=session)
exp.set_experiment("User_activity_forecasting_models")

#### Get training dataset from FS

In [ ]:
# Connect to existing Feature Store
fs = FeatureStore(
    session=session,
    database="dev",
    name="user_activity_feature_store",
    default_warehouse="analytics_wh_flex"
)

In [ ]:
# Retrieve required feature views
fv_ads = fs.get_feature_view(name="user_features_ads", version="1.2")
fv_call_quality = fs.get_feature_view(name="user_features_call_quality", version="1.2")
fv_call_rating = fs.get_feature_view(name="user_features_call_rating", version="1.2")
fv_data_usage = fs.get_feature_view(name="user_features_data_usage", version="1.2")
fv_drawer_event = fs.get_feature_view(name="user_features_drawer_event", version="1.2")
fv_inbound_calls = fs.get_feature_view(name="user_features_inbound_calls", version="1.2")
fv_messages = fs.get_feature_view(name="user_features_messages", version="1.2")
fv_nps_rating = fs.get_feature_view(name="user_features_nps_rating", version="1.2")
fv_outbound_calls = fs.get_feature_view(name="user_features_outbound_calls", version="1.2")
fv_redial = fs.get_feature_view(name="user_features_redial", version="1.2")
fv_sessions = fs.get_feature_view(name="user_features_sessions", version="1.2")

all_features = [
    fv_ads,
    fv_call_quality, 
    fv_call_rating,
    fv_data_usage,
    fv_drawer_event,
    fv_inbound_calls,
    fv_messages,
    fv_nps_rating,
    fv_outbound_calls,
    fv_redial,
    fv_sessions  
]

In [ ]:
# spine df
spine_df = session.sql("""
WITH daily AS (
    SELECT username, date_utc, SUM(time_in_app_mins_per_day) AS mins
    FROM metrics_daily_userlevel_app_time_sessions
    WHERE date_utc >= DATEADD('day', -37, '2025-11-19')
    GROUP BY username, date_utc
),
labeled AS (
    SELECT up.user_id_hex, d.date_utc,
        COALESCE(SUM(IFF(d.mins > 1, 1, 0)) OVER (
            PARTITION BY d.username ORDER BY d.date_utc
            RANGE BETWEEN INTERVAL '1 DAY' FOLLOWING AND INTERVAL '7 DAYS' FOLLOWING
        ), 0) AS active_days_in_week
    FROM daily d
    JOIN user_profiles up ON d.username = up.latest_username
),
filtered AS (
    SELECT * FROM labeled
    WHERE date_utc <= DATEADD('day', -7, '2025-11-19')
),
sampled AS (
    SELECT *, ROW_NUMBER() OVER (
        PARTITION BY date_utc 
        ORDER BY HASH(user_id_hex || date_utc)
    ) AS rn
    FROM filtered
)
SELECT user_id_hex, date_utc, active_days_in_week 
FROM sampled 
WHERE rn <= 70000
""")

In [ ]:
#Generate training dataset
df = fs.generate_training_set(
    spine_df=spine_df,
    features=all_features,
    spine_label_cols="active_days_in_week",
)

In [ ]:
# to_pandas
df = df.to_pandas().fillna(0)

In [ ]:
df.head()

## Data Exploration

In [ ]:
pd.set_option('display.max_columns', None)
def explore_data(dataframe): 
    # Display first few rows of the dataset
    print(dataframe.head()) 

    print("\nData Types of Each Column:")
    print(dataframe.dtypes)
    
    print("\nStatistics of the Numerical Columns:")
    print(dataframe.describe())
    
    print("\nMissing Values in Each Column:")
    print(dataframe.isnull().sum())
explore_data(df)

In [ ]:
# Function to plot variable distributions
def plot_variable_distributions(dataframe):
    """Plot histograms for all numeric features"""
    # Get numeric columns
    numeric_columns = dataframe.select_dtypes(include=['number']).columns
    
    # Calculate grid size
    num_features = len(numeric_columns)
    num_cols = 3
    num_rows = (num_features + num_cols - 1) // num_cols
    
    # Create subplots
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, num_rows * 4))
    axes = axes.flatten()
    
    # Plot each feature
    for i, column_name in enumerate(numeric_columns):
        axes[i].hist(dataframe[column_name].dropna(), bins=30, 
                    color='skyblue', edgecolor='black', alpha=0.7)
        axes[i].set_title(column_name, fontsize=10)
        axes[i].set_xlabel(column_name)
        axes[i].set_ylabel('Frequency')
        axes[i].grid(alpha=0.3)
    
    # Hide empty subplots
    for i in range(num_features, len(axes)):
        axes[i].axis('off')
    
    plt.suptitle("Variable Distributions", fontsize=15, y=1.00)
    plt.tight_layout()
    plt.show()

# Call the function to plot distributions
plot_variable_distributions(df)

In [ ]:
# Correlation matrix
def plot_correlation_matrix(dataframe, top_n=20):
    """Plot heatmap showing correlations between features"""
    numeric_data = dataframe.select_dtypes(include=['number'])
    correlation_matrix = numeric_data.corr()
    
    # Get top N features most correlated with target
    target_corr = correlation_matrix['ACTIVE_DAYS_IN_WEEK'].abs().sort_values(ascending=False)
    top_features = target_corr.head(top_n).index.tolist()
    
    # Filter correlation matrix to only top features
    filtered_corr = correlation_matrix.loc[top_features, top_features]
    
    # Create heatmap
    plt.figure(figsize=(14, 12))
    sns.heatmap(filtered_corr, annot=True, fmt='.2f', cmap='coolwarm',
                center=0, square=True, linewidths=0.5, 
                cbar_kws={"shrink": 0.8},
                annot_kws={"size": 8})
    plt.title(f'Top {top_n} Features - Correlation Matrix', fontsize=16, pad=20)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    # Print correlations with target
    print("\nCorrelation with Target (ACTIVE_DAYS_IN_WEEK):")
    print("=" * 60)
    for feature, corr_value in target_corr.head(top_n).items():
        print(f"{feature}: {corr_value:.4f}")

plot_correlation_matrix(df, top_n=15)

In [ ]:
## Feature Engineering and Preparation

In [ ]:
# Split training and testing dataset
X = df.drop(['USER_ID_HEX', 'DATE_UTC', 'ACTIVE_DAYS_IN_WEEK'], axis=1)
y = df['ACTIVE_DAYS_IN_WEEK']

In [ ]:
## Feature selection
def remove_low_variance_features(dataframe, variance_threshold=0.01):
    # Create variance selector
    variance_selector = VarianceThreshold(threshold=variance_threshold)
    variance_selector.fit(dataframe)
    
    # Get feature names that pass the threshold
    selected_features = dataframe.columns[variance_selector.get_support()].tolist()
    removed_features = dataframe.columns[~variance_selector.get_support()].tolist()
    
    # Display results
    print(f"Variance threshold: {variance_threshold}")
    print(f"Total features to remove: {len(removed_features)}")
    
    if removed_features:
        print("\nFeatures being removed (low variance):")
        for feature in removed_features:
            feature_variance = dataframe[feature].var()
            print(f"  - {feature} (variance: {feature_variance:.6f})")
    else:
        print("No low variance features found!")
    
    # Create dataframe with selected features
    dataframe_after_removal = dataframe[selected_features]
    
    print(f"\nFeatures before: {dataframe.shape[1]}")
    print(f"Features after: {dataframe_after_removal.shape[1]}")
    print("=" * 60)
    
    return dataframe_after_removal, removed_features

# Remove low variance features from X
X = remove_low_variance_features(X, variance_threshold=0.01)[0]

In [ ]:
print(f"Features after variance filter: {list(X.columns)}")

In [ ]:
# Temporal split: 80% of days for training, 20% for testing
dates = df['DATE_UTC']
unique_dates = sorted(dates.unique())
n_train_days = int(len(unique_dates) * 0.8)
cutoff_date = unique_dates[n_train_days - 1]

train_mask = dates <= cutoff_date
test_mask = dates > cutoff_date

X_train = X[train_mask].reset_index(drop=True)
X_test = X[test_mask].reset_index(drop=True)
y_train = y[train_mask].reset_index(drop=True)
y_test = y[test_mask].reset_index(drop=True)

print(f"Train: {len(X_train)} rows ({n_train_days} days, <= {cutoff_date})")
print(f"Test:  {len(X_test)} rows ({len(unique_dates) - n_train_days} days, > {cutoff_date})")

In [ ]:
# Preprocessing
numerical_cols = X.select_dtypes(include=['number']).columns
preprocess=ColumnTransformer([
    ("num", StandardScaler(), numerical_cols)
])

In [ ]:
# Hyperparameter tuning
print(f"Hyper Parameter Tuning with RandomizedSearch")
models = [
    # Linear models
    ("LinearRegression", LinearRegression(), {
        "model__fit_intercept": [True, False],
        "model__positive": [False, True],
    }),
    ("Ridge", Ridge(), {
        "model__alpha": [1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0, 1000.0],
        "model__fit_intercept": [True, False],
        "model__solver": ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"],
        "model__tol": [1e-4, 1e-3, 1e-2],
        "model__max_iter": [1000, 5000],
    }),
    ("Lasso", Lasso(random_state=42), {
        "model__alpha": [1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0],
        "model__fit_intercept": [True, False],
        "model__selection": ["cyclic", "random"],
        "model__tol": [1e-4, 1e-3, 1e-2],
        "model__max_iter": [2000, 5000, 10000],
    }),
    
    # Random Forest
    ("RandomForest", RandomForestRegressor(random_state=42), {
        "model__n_estimators": [200, 500, 800],
        "model__max_depth": [None, 10, 20, 40],
        "model__min_samples_split": [2, 5, 10],
        "model__min_samples_leaf": [1, 2, 4],
        "model__max_features": ["sqrt", 0.5, 1.0],
        "model__bootstrap": [True, False],
    }),
    
    # Gradient Boosting
    ("GradientBoosting", GradientBoostingRegressor(random_state=42), {
        "model__n_estimators": [200, 500, 800],
        "model__learning_rate": [0.01, 0.03, 0.05, 0.1],
        "model__max_depth": [2, 3, 4, 5],
        "model__subsample": [0.6, 0.8, 1.0],
        "model__min_samples_split": [2, 5, 10],
        "model__min_samples_leaf": [1, 2, 4],
        "model__max_features": [None, "sqrt"],
    }),
    
    # XGBoost
    ("XGBoost", XGBRegressor(
        verbosity=0,
        random_state=42,
        objective="reg:squarederror",
        tree_method="hist",
        n_jobs=-1,
    ), {
        "model__n_estimators": [300, 600, 1000],
        "model__learning_rate": [0.01, 0.03, 0.05, 0.1],
        "model__max_depth": [3, 4, 6, 8],
        "model__subsample": [0.6, 0.8, 1.0],
        "model__colsample_bytree": [0.6, 0.8, 1.0],
        "model__min_child_weight": [1, 5, 10],
        "model__gamma": [0.0, 0.1, 0.5, 1.0],
        "model__reg_alpha": [0.0, 1e-3, 1e-2, 0.1, 1.0],
        "model__reg_lambda": [0.1, 1.0, 10.0],
    }),
    
    # LightGBM
    ("LightGBM", LGBMRegressor(
        random_state=42,
        verbosity=-1,
        n_jobs=-1,
    ), {
        "model__n_estimators": [300, 600, 1000],
        "model__learning_rate": [0.01, 0.03, 0.05, 0.1],
        "model__max_depth": [-1, 5, 8, 12],
        "model__num_leaves": [31, 63, 127, 255],
        "model__min_child_samples": [10, 20, 50, 100],
        "model__subsample": [0.6, 0.8, 1.0],
        "model__colsample_bytree": [0.6, 0.8, 1.0],
        "model__reg_alpha": [0.0, 1e-3, 1e-2, 0.1, 1.0],
        "model__reg_lambda": [0.1, 1.0, 10.0],
    })
]

# Track only the best model.
best_result = {
    "name": None,
    "rmse": np.inf,
    "mae": np.inf,
    "best_params": {},
    "pipeline": None,
    "training_time_seconds": None,
}

for name, model, param_grid in models:
    print(f"Training with RandomizedSearch: {name}")

    pipeline = Pipeline(steps=[
        ("preprocess", preprocess),
        ("model", model)
    ])

    random_search  = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_grid,
        n_iter=25,
        cv=TimeSeriesSplit(n_splits=3),
        scoring="neg_root_mean_squared_error",
        n_jobs=1,
        pre_dispatch=1,
        verbose=1,
        random_state=42
    )
    
    # Start timer
    start_time= time.time()
    
    # Fit on training data
    random_search.fit(X_train, y_train)
    
    # Calculate training time
    training_time = time.time() - start_time
    
    candidate_pipeline = random_search.best_estimator_
    candidate_params = random_search.best_params_
    y_pred = candidate_pipeline.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    
    # Get sizes
    n_train = len(X_train)
    n_test= len(X_test)
    n_features = X_train.shape[1]
    
    print(f"{name} -> Best Params: {candidate_params}")
    print(f"{name} -> Test RMSE: {rmse:.4f}")
    print(f"{name} -> Test MAE: {mae:.4f}")

    # Keep only the current best pipeline in memory.
    if rmse < best_result["rmse"]:
        best_result["pipeline"] = None
        best_result = {
            "name": name,
            "rmse": float(rmse),
            "mae": float(mae),
            "best_params": dict(candidate_params),
            "pipeline": candidate_pipeline,
            "training_time_seconds": float(training_time),
            "n_train_samples": int(n_train),
            "n_test_samples": int(n_test),
            "n_features": int(n_features),
        }
    else:
        candidate_pipeline = None
    

# Print + log the best model.
if best_result["pipeline"] is not None:
    print("\nBest model (by RMSE):")
    print(f"  Model: {best_result['name']}")
    print(f"  RMSE:  {best_result['rmse']:.4f}")
    print(f"  MAE:   {best_result['mae']:.4f}")
    print(f"  Params: {best_result['best_params']}")

    with exp.start_run():
        exp.log_metric("rmse", best_result["rmse"])
        exp.log_metric("mae", best_result["mae"])
        exp.log_metric("training_time_seconds", best_result["training_time_seconds"])
        exp.log_param("model_type", best_result["name"])
        exp.log_param("n_train_samples", best_result["n_train_samples"])
        exp.log_param("n_test_samples", best_result["n_test_samples"])
        exp.log_param("n_features", best_result["n_features"])
        for param_name, param_value in best_result["best_params"].items():
            exp.log_param(param_name, param_value)
        exp.log_model(
            model=best_result["pipeline"],
            model_name=f"{best_result['name']}_best_model",
            sample_input_data=X_train.head(),
        )

In [ ]:
## Feature Importance Analysis

In [ ]:
# Get feature importance
pipeline= best_result["pipeline"]
model = pipeline.named_steps['model']
feature_importance = model.feature_importances_
feature_names = X.columns

# Create explainability report
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("Feature Importance Rankings:")
print(importance_df)

In [ ]:
# SHAP
# Get transform data
X_train_transformed = pipeline.named_steps['preprocess'].transform(X_train)

# Sample for speed
X_sample = X_train_transformed[:1000]

# SHAP
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_sample)

# Global SHAP importance = mean absolute SHAP
mean_abs_shap = np.mean(np.abs(shap_values), axis=0)

# Plot 1: Beeswarm (detailed)
plt.figure()
shap.summary_plot(shap_values, X_sample, feature_names=X_train.columns, show=False)
plt.tight_layout()
plt.show()

# Plot 2: Bar (simple importance)
plt.figure()
shap.summary_plot(shap_values, X_sample, feature_names=X_train.columns, plot_type="bar", show=False)
plt.tight_layout()
plt.show()

In [ ]:
NUMERICAL_NOISE_EPS = 0.01
CORRELATION_THRESHOLD = 0.9

# Build a report table
feature_report = pd.DataFrame({
    "feature": feature_names,
    "feature_importance": feature_importance,
    "mean_abs_shap": mean_abs_shap
}).sort_values(["mean_abs_shap", "feature_importance"], ascending=False)

print("Top features (SHAP then importance):")
print(feature_report.head(15))

# keep feature if either signal is non-zero
feature_report["keep"] = (
    (feature_report["feature_importance"] > NUMERICAL_NOISE_EPS) |
    (feature_report["mean_abs_shap"] > NUMERICAL_NOISE_EPS)
)

selected_features = feature_report.loc[feature_report["keep"], "feature"].tolist()
dropped_features_zero_signal = feature_report.loc[~feature_report["keep"], "feature"].tolist()

print("\nSelected features (before correlation):", len(selected_features))
print("\nSelected features (before correlation) list:", selected_features)
print("Dropped features (zero signal):", len(dropped_features_zero_signal))
print("Dropped (zero signal) list:", dropped_features_zero_signal)

## Correlation pruning ONLY among selected features

# feature -> SHAP strength
shap_strength_by_feature = dict(
    zip(feature_report["feature"], feature_report["mean_abs_shap"])
)

# Correlation among selected features
correlation_matrix = X_train[selected_features].corr().abs()

# Find correlated feature pairs (upper triangle only)
row_idx, col_idx = np.where(
    np.triu(correlation_matrix.values, k=1) > CORRELATION_THRESHOLD
)

features_to_drop_due_to_correlation = set()
correlation_feature_names = correlation_matrix.columns


for i, j in zip(row_idx, col_idx):
    feature_a = correlation_feature_names[i]
    feature_b = correlation_feature_names[j]

    # Drop the weaker feature (smaller SHAP contribution)
    if shap_strength_by_feature[feature_a] >= shap_strength_by_feature[feature_b]:
        features_to_drop_due_to_correlation.add(feature_b)
    else:
        features_to_drop_due_to_correlation.add(feature_a)

final_selected_features = [
    f for f in selected_features if f not in features_to_drop_due_to_correlation
]

print("\nDropped due to correlation:", len(features_to_drop_due_to_correlation))
print("Dropped (correlation) list:", sorted(features_to_drop_due_to_correlation))
print("Final selected features:", len(final_selected_features))
print("Final feature list:", final_selected_features)

## Next Steps
The selected features can be used to retrain a more focused model or update the production config.

In [ ]:
print(f"Final selected features: {final_selected_features}")